# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
This is a classification problem because given the information for each student, we want to teach a machine to understand what students are going to fail to graduate. Since the output is represented by "passed", which is a discrete variable that can only be "true" or "false", it is a classification problem. Instead, regressions are used to predict a numeric value in a continuous range.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [2]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [3]:
# TODO: Calculate number of students
n_students = len(student_data.index)

# TODO: Calculate number of features
n_features = len(student_data.columns.difference(['passed']))

# TODO: Calculate passing students
n_passed = len(student_data[student_data['passed'] == 'yes'])

# TODO: Calculate failing students
n_failed = len(student_data[student_data['passed'] == 'no'])

# TODO: Calculate graduation rate
grad_rate = 100.0*n_passed/n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [4]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [5]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [6]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=num_train, test_size=num_test, random_state=27, stratify=y_all)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

**RANDOM FOREST (Ensemble Methods)**

***Real world application:***  Random forests have been used to effectively learn non-linear relationships between forest vegetation and biophysical factors, in order to classify various ecosystems as different forests types.
http://www.mdpi.com/2072-4292/5/6/2838/htm

***Strenghts of the model:*** Very good at handling categorical features, since it doesn't expect linear features. Very fast to train.

***Weaknesses:*** Slow to create predictions once trained. Less accurate than Boosting. Interpretability is an issue.

***What makes it a good candidate for the problem:*** Random forests are very good and often used in classification problems, which is the case. Also they're very good at evading overfitting, something we don't want to have when dealing with this problem either. We also have lots of categorical features in our dataset, an input that the Random Forest is made to handle.

**SUPPORT VECTOR MACHINES**

***Real world application:*** One application of SVMs in industry is in the field of Neuroimage analysis. They have been found to be an effective multivariate analysis tool for group comparison. It allows to quantify structural and functional differences between diverse groups of patients and detect brain states associated with a stimulus.
Yunqian and Guo, G. (eds.) (2014) Support vector machines applications. Switzerland: Springer International Publishing AG.

***Strenghts of the model:*** It is effective in high dimensional spaces. It is memory efficient. It can classify non-linearly separable classes. It has a regularization parameter that allows to avoid overfitting.

***Weaknesses:*** If the number of features is much better than the amount of samples, it is likely to perform badly. Choosing the correct kernel can be tricky. Non standarized approach to deal with multi-class classification problems. Results are hard to understand.

***What makes it a good candidate for the problem:*** SVMs perform well at classification problems. The amount of dimensions of the data (47) is much smaller than the amount of samples in the input (395). They can classify non-linearly separable data, which might be the case.

**GRADIENT BOOSTING (Ensemble Methods)**

***Real world application:*** Gradient Boosting have can used as a regression model in order to predict the CTR (Click Through Rate) of websites, given a Click Through Log. Using this model could increase the profits for advertisers. http://link.springer.com.ezproxy.uniandes.edu.co:8080/chapter/10.1007/978-3-319-46257-8_15

***Strenghts of the model:*** Every new weak learner is built on top of the previous one, which helps to correct previous errors. Can outperform Random Forests if parameters are correctly set.

***Weaknesses:***  Hard to tune. There are many hyperparameters. It can easiy overfit. Slow to train. There's no straightforward way to understand how variables contribute to final prediction.

***What makes it a good candidate for the problem:*** Gradient Boosting can solve ML classification problems such as the given one. It can detect non-linear feature interactions, which might be the case for this problem.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [7]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))
    print ""

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [8]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

# TODO: Initialize the three models
my_random_state = 3
clf_A = RandomForestClassifier(random_state=my_random_state)
clf_B = SVC(random_state=my_random_state)
clf_C = GradientBoostingClassifier(random_state=my_random_state)

# TODO: Execute the 'train_predict' function for each classifier and each training set size

for clf in [clf_A, clf_B, clf_C]:
    
    # show the estimator type
    print "\n{}: \n".format(clf.__class__.__name__)

    # TODO: Set up the training set sizes
    # loop thru training sizes
    for n in [100, 200, 300]:
        
        # create training data from first 100, then 200, then 300 points
        train_predict(clf, X_train[:n], y_train[:n], X_test, y_test)


RandomForestClassifier: 

Training a RandomForestClassifier using a training set size of 100. . .
Trained model in 0.0415 seconds
Made predictions in 0.0014 seconds.
F1 score for training set: 0.9927.
Made predictions in 0.0018 seconds.
F1 score for test set: 0.7755.

Training a RandomForestClassifier using a training set size of 200. . .
Trained model in 0.0708 seconds
Made predictions in 0.0017 seconds.
F1 score for training set: 0.9925.
Made predictions in 0.0015 seconds.
F1 score for test set: 0.6667.

Training a RandomForestClassifier using a training set size of 300. . .
Trained model in 0.0324 seconds
Made predictions in 0.0027 seconds.
F1 score for training set: 0.9800.
Made predictions in 0.0015 seconds.
F1 score for test set: 0.6923.


SVC: 

Training a SVC using a training set size of 100. . .
Trained model in 0.0017 seconds
Made predictions in 0.0009 seconds.
F1 score for training set: 0.8774.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.8205.

Training a S

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Random Forest Classifier**  

| Training Set Size |      Training Time      | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |          0.0440         |         0.0013         |      0.9927      |     0.7755      |
| 200               |          0.0291         |         0.0014         |      0.9925      |     0.6667      |
| 300               |          0.0271         |         0.0014         |      0.9800      |     0.6923      |

** Classifer 2 - SVC**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |          0.0013         |         0.0007         |      0.8774      |     0.8205      |
| 200               |          0.0042         |         0.0016         |      0.8636      |     0.8052      |
| 300               |          0.0083         |         0.0017         |      0.8584      |     0.7651      |

** Classifer 3 - Gradient Boosting**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |          0.0757         |         0.0005         |      1.0000      |     0.8000      |
| 200               |          0.0916         |         0.0005         |      0.9925      |     0.7424      |
| 300               |          0.1043         |         0.0006         |      0.9757      |     0.7143      |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
The model that is most appropiate for solving this problem is the SVC. This is because total cost in time of training and predicting the data is much lower than the cost of doing so with any other model. Additionally, it is the model that overfits the least; the two other models are better when predicting the training data, but perform worse tan the SVC when proedicting the testing data. SVMs perform really good when the amount of data is not too big, just like the current case.

Nevertheless, if we compare the results for all three models generated prevously with a benchmark of predicting all 'yes' on the test set, we will find out that the F1 scores gotten are worst than the benchmark:

In [9]:
from sklearn.metrics import f1_score
print "F1 score for predicting all \"yes\" on test set: {:.4f}".format(
    f1_score(y_test, ['yes']*len(y_test), pos_label='yes', average='binary'))


F1 score for predicting all "yes" on test set: 0.8050


### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **


Support Vector Machines are a model that is used to learn from data. When predicting discrete values, Support Vector Machines find out which are the datapoints that "represent" the separation between two different target classes of points. These datapoints are then used to find the line that separates the best both clusters, in such a way that the following predictions will have the highest accuracy possible:

![](img/svm1.jpg) *Image source: http://diggdata.in/post/94066544971/support-vector-machine-without-tears*

In order to make a prediction, the model verifies in which side of the line the new datapoint is. Because the model already knows which side belongs to which class, it just needs to assign the class found in that area.

Sometimes it is not too easy to separate the data with just a straight line; it might be better to have a curved line that separates the data better. Support Vector Machines are able to use curved lines instead to separate the data, as shown in the following image:

![](img/svm2.png) *Image source: http://www.eric-kim.net/eric-kim-net/posts/1/kernel_trick.html*

Finally, Support Vector Machines can learn very well to classify the datapoints used to learn, generating complex representations. Neverthelees this might worsen the accuracy when predicting new datasets. Because of this, SVMs can be set to misclassify some datapoints when learning, in order to generate simpler models that can extrapolate to new predictions:

![](img/svm3.png) *Image source: https://www.quora.com/What-are-the-advantages-of-support-vector-machines-SVM-compared-with-linear-regression-or-logistic-regression*

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [14]:
# TODO: Import 'GridSearchCV' and 'make_scorer'

from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

# TODO: Create the parameters list you wish to tune
parameters = [
  {'C': [1,10,100,1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
  {'C': [1, 10, 100, 1000], 'degree' : [1,3,5,10], 'gamma': [0.001, 0.0001], 'kernel': ['poly']}
]
# TODO: Initialize the classifier
clf = SVC()
# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")
# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)
# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)
# Get the estimator
clf = grid_obj.best_estimator_

print grid_obj.best_params_
# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

{'kernel': 'rbf', 'C': 100, 'gamma': 0.0001}
Made predictions in 0.0049 seconds.
Tuned model has a training F1 score of 0.8369.
Made predictions in 0.0017 seconds.
Tuned model has a testing F1 score of 0.7724.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
The final model's F1 score for training is 83.69% and for testing is 77.24%.

This model is much better at predicting the testing data than the untuned model, by a difference of 0.73%. It is also better at predicting testing data, rather than training data. 

It also took the same time that the previous model took to make predictions. This means that it develops slightly better given the same conditions.

In conclusion, it performs better than the untuned model, nevertheless it is still worse than the benchmark generated, which means there are more and simpler models to predict new student data.

Following you will find all the different scores obtained for every combination of parameters in the grid search done.


In [15]:
from IPython.display import display
display(pd.DataFrame(grid_obj.grid_scores_))

,parameters,mean_validation_score,cv_validation_scores
0,"{u'kernel': u'linear', u'C': 1}",0.761526,"[0.732824427481, 0.743243243243, 0.808510638298]"
1,"{u'kernel': u'linear', u'C': 10}",0.753740,"[0.71875, 0.756756756757, 0.785714285714]"
2,"{u'kernel': u'linear', u'C': 100}",0.752252,"[0.714285714286, 0.756756756757, 0.785714285714]"
3,"{u'kernel': u'linear', u'C': 1000}",0.755627,"[0.724409448819, 0.756756756757, 0.785714285714]"
4,"{u'kernel': u'rbf', u'C': 1, u'gamma': 0.001}",0.810490,"[0.807228915663, 0.812121212121, 0.812121212121]"
5,"{u'kernel': u'rbf', u'C': 1, u'gamma': 0.0001}",0.802395,"[0.802395209581, 0.802395209581, 0.802395209581]"
6,"{u'kernel': u'rbf', u'C': 10, u'gamma': 0.001}",0.811574,"[0.787096774194, 0.810126582278, 0.8375]"
7,"{u'kernel': u'rbf', u'C': 10, u'gamma': 0.0001}",0.805618,"[0.807228915663, 0.802395209581, 0.807228915663]"
8,"{u'kernel': u'rbf', u'C': 100, u'gamma': 0.001}",0.775687,"[0.735294117647, 0.765100671141, 0.826666666667]"
9,"{u'kernel': u'rbf', u'C': 100, u'gamma': 0.0001}",0.814225,"[0.792452830189, 0.807453416149, 0.842767295597]"


> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.